Notes

In [8]:
import sympy as sp
import matplotlib.pyplot as plt
%matplotlib inline


# initiate pretty print
sp.init_printing()

# define parameter
t = sp.Symbol('t')

Define polynomial

In [9]:
my_polynomial = t**2 + 1
my_polynomial

Plot the polynomial and find roots (solve p for t), including multiplicities

In [10]:
#ax = sp.plot(my_polynomial, xlim=(-2, 2), ylim=(0,4), aspect_ratio=(1,1))

roots = sp.roots(my_polynomial)
roots

Hankel matrix of polynomial

In [11]:
p = sp.Poly(my_polynomial, t, domain='RR')
coeffs = p.all_coeffs()

deg = p.degree()

In [12]:
# companion matrix
companion_matrix = sp.zeros(deg)
for i in range(deg - 1):
    companion_matrix[i + 1, i] = 1
for i in range(deg):
    companion_matrix[i, -1] = -1 * coeffs[::-1][i]

newton_sums = []
for k in range(2*deg - 1):
    V_k = companion_matrix ** k
    newton_sums.append(V_k.trace())

newton_sums

In [13]:
# construct the Hankel matrix
hankel_matrix = sp.zeros(deg)
for i in range(deg):
    for j in range(deg):
        hankel_matrix[i, j] = newton_sums[i+j]

hankel_matrix

⎡2   0  ⎤
⎢       ⎥
⎣0  -2.0⎦

In [14]:
eigenvalues = hankel_matrix.eigenvals()

# Count the positive and negative eigenvalues
positive_eigenvalues = sum(1 for eigenvalue in eigenvalues if eigenvalue > 0)
negative_eigenvalues = sum(1 for eigenvalue in eigenvalues if eigenvalue < 0)

# Calculate the signature
signature = positive_eigenvalues - negative_eigenvalues
rank = hankel_matrix.rank()

Rank of the matrix:

In [15]:
rank

Signature of the matrix:

In [16]:
signature

Roots again:

In [17]:
roots